# HW2a: Kotlin Calculator
- 이름: ???
- 학번: ????????

https://calculatorhistory.net

### Define your `Calculator` class by overriding abstract methods of the `ICalculator` interface (??점)

`ICalculator` 인터페이스의 주석에 지시된 내용과 수업시간에 설명한 내용을 바탕으로 해당 인터페이스를 구현하는 `Calculator`를 작성하고 적절히 테스트하라.

`appendEntry` 메소드의 주석에 "legal entry"라는 표현이 등장하는데, TODO ????

Calculator의 기능을 구현하기 위해서 적절한 인스턴스 변수들도 선언되어야 할 것이다.

### Override `Calculator::toString` (10점)

테스트 출력을 위해서는 적절히 보기좋게 계산기의 상태를 출력할 필요가 있다. 따라서 주어진 ICalculator 인터페이스의 추상 메소드를 오버라이딩하는 것 외에도 출력을 위한 `toString`도 적절히 오버라이드하라.

예컨대, `1 + 1`을 계산한 다음 `2 * 3`을 계산하고 나서 지금은 미완성인 식 `(`를 입력중인 상황이라면
Calculator 객체가 **정확히** 다음과 같이 출력되어야 한다.
들여쓰기에 해당하는 왼쪽의 빈칸들은 space가 아닌 tab으로 처리하라.

```
Calculator[
	entries="(",
	currVal=null,
	lastVal=6,
	history=[
		2 * 3 = 6,
		1 + 1 = 2
	]
]
```

Calculator의 기능을 완전히 완성하지 않더라도 이것만으로 부분점수가 부여될 수 있다.
예를 들면 `new Calculator();`만 실행한 상태에서 history나 entries등을 빈 리스트 등의 정상적인 기본값이 아닌
테스트를 위해 임의로 초기화시켜 놓고 `toString`이 작동한다는 것을 시연하는 테스트가 있으면 부분점수 획득이 가능하다.
당연히 `toString`에 위 내용을 문자열로 하드코딩하면 당연히 인정이 안되고, 인스턴스 변수에 있는 데이터로부터 처리하여 문자열을 만들 내야 부분점수로 인정될 요건이 된다.

자신이 작성한 프로그램이 잘 동작하는지 알아보기 위해서는 테스트를 실행했을 때 적절한 데이터가 출력되는지 알아보아야 하는 기능이 필수적이다.
그러니까 작성하기 쉬운 간단한 메소드들은 먼저 작성해 놓더라도 조금 복잡하다고 생각되는 것을 작성하여 모든 내용을 완성하기 전에 일단 `toString`부터 작성할 것을 권장한다.

----

In [1]:
sealed interface Entry {
    enum class BinOp: Entry { ADD, MULT }
    enum class Delim: Entry { LPAREN, RPAREN }
    data class Value(val number: Int): Entry
}

In [2]:
Entry.BinOp.ADD

ADD

In [3]:
Entry.Delim.LPAREN

LPAREN

In [4]:
Entry.Value(43)

Value(number=43)

In [5]:
fun asString(e: Entry) = when (e) {
    Entry.BinOp.ADD    -> "+"
    Entry.BinOp.MULT   -> "*"
    Entry.Delim.LPAREN -> "("
    Entry.Delim.RPAREN -> ")"
    is Entry.Value -> e.number.toString()
}

In [6]:
asString( Entry.BinOp.ADD )

+

In [7]:
asString( Entry.Value(43) )

43

In [8]:
val v1 = Entry.Value(43);
val v2 = Entry.Value(56);
val es = listOf(v1, Entry.BinOp.MULT, Entry.Delim.LPAREN, v2, Entry.BinOp.ADD, v1, Entry.Delim.RPAREN );

println(es)

println( es.map(::asString) )

es.map(::asString).reduce { x, y -> x + " " + y }

[Value(number=43), MULT, LPAREN, Value(number=56), ADD, Value(number=43), RPAREN]
[43, *, (, 56, +, 43, )]


43 * ( 56 + 43 )

In [9]:
interface ICalculator {
    // Adds an entriy (either BinOp, Delim, or Vaule) at the end.
    // Appends legal entry input and return true (returns false for illegal input without adding it).
    // Further details are discussed in the homework assignment instruction above.
    fun appendEntry(e: Entry): Boolean

    // Removes the last added entry; The ◄(x or DEL) button
    // Removes the last input entry and returns true (returns false if not possible)
    fun removeEntry(): Boolean
    
    fun currValue(): Entry.Value? // currently calculated value from the current entries (defaults to null)
    fun lastValue(): Entry.Value  // last calculated value (defaults to 0)
    fun entries(): List<Entry> // get current entries
    fun history(): List< Pair<List<Entry>, Entry.Value> > // history of calculations
    fun clearEntries() // clear the current entries; AC button
    fun clearHistory() // clear history

    // The = button.
    // Adds the current entries and its calculated current value as a pair into history,
    // when the current entries form a valid expression to calculate the current vaule,
    // and clears the current entries and resets the current value.
    // Returns the calculated result value for adding valid form of entries and its result value.
    // Returns null, without adding to the history, when the current entries do not form a valid expression.
    fun calculate(): Entry.Value?
}

----
참고자료: https://madplay.github.io/post/call-javascript-function-from-java-using-graalvm

In [10]:
@file:DependsOn("org.graalvm.js:js:22.0.0.2")
@file:DependsOn("org.graalvm.js:js-scriptengine:22.0.0.2")
import org.graalvm.polyglot.*

In [11]:
val context = Context.create("js"); // 그냥 항상 성공한다고 가정하면 이렇게 써도 ...

val value = context.eval("js", "3 + 4 * 10");
println( value );
println( value.toString() == "43" )
println( value::class )

val vi: Int = value.asInt()
vi == 43

43
true
class org.graalvm.polyglot.Value


true

In [12]:
val context = Context.create("js"); // 그냥 항상 성공한다고 가정하면 이렇게 써도 ...

try {
    val value = context.eval("js", "3 + 4 * ");
    println( value );
    println( value.toString() == "43" )
    println( value::class )
} catch (e : Exception) {
    println(e)
    println("이걸 적당히 잘 처리하면 유용하게 활용 가능")
}

org.graalvm.polyglot.PolyglotException: SyntaxError: Unnamed:1:8 Expected an operand but found eof
3 + 4 * 
        ^

이걸 적당히 잘 처리하면 유용하게 활용 가능


----

In [13]:
// ... class Calculator: ICalculator ...